#Assignment 1 (Random Forest)

* In this project you are given a dataset of housing housing price prediction. 
Dataset description is found in the given datasets.

* The goal of the project is to predict the price of a house given its attributes. 
Therefore, the problem is a regression task. 

* You need to build a random forest that consists of multiple decision trees (for regression) from the given training data set. Then, apply it on the test set and submit your code to generate predictions.
You need to build the random forest and decision trees from scratch. (I.e., it is not allowed to use existing machine learning libraries or packages such as sklearn.)

* You may use any programming language/environment of your choice, but you are required to submit the complete source code to produce the output
If you use anything other than jupyter notebook, submit an executable and run that from the main function of the jupyter notebook so that the prediction generation is automated. We can provide assistance with this.
The output (a single file with the predictions for each test instance) must be generated automatically using the approach implemented by you. Submitting predictions/code from any other source (Internet, another student, etc.) is considered cheating and will result in immediate disqualification (i.e., dismissal from the course).

##Part 1: Preprocessing and dataset analysis (20 points)

* The given dataset is quite complex, it has many attributes, and not all of them are useful! 
Training on such dataset results in a bad accuracy. And this is exactly the point! 

* "Understanding the question is half the answer". In data mining, understanding the dataset is half the answer! 

* In part 1 you need to analyze the dataset and make it clean. 
 

###Load the dataset and explore (5 points)

* Load the dataset, view the dataset and the shape of it, 

In [28]:
import pandas as pd

# Load the training and test datasets. 
# NA values are given the value of 0 aka null
# consider poor as value of 1
train_df = pd.read_csv('housing_price_train.csv', na_values='NA')
test_df = pd.read_csv('housing_price_test.csv', na_values='NA')

# Display basic information about the datasets
print("----- Viewing the training dataset -----")
print(train_df.head())
#Header already shows the shape
#print(f"Shape: {test_df.shape} (rows, columns)")


print("\n----- Viewing the test dataset -----")
print(test_df.head())


# Check for missing values (example for training data)
print("\n=== Missing Values in Training Data ===")
list_of_null= train_df.isnull().sum()
print(list_of_null[list_of_null> 0])

----- Viewing the training dataset -----
   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCon

### Clean the dataset (10 points)

* We cannot train on a 'dirty' dataset! There are duplicated, Null, and missing values that you need to take care of!

* Drop all columns which have null values >= 70 % and drop all rows which have null values >= 70 %.

* You need to take care of categorial data!

In [1]:
import pandas as pd
def PreprocessingData(db):
    """
    Preprocess the dataset:
    - Show NULL values for each column and their percentage.
    - Drop all columns with null values >= 70%.
    - Drop all rows with null values >= 70%.
    - Fill missing values with the mean (numerical) or mode (categorical).
    """
 # Create a copy to avoid modifying the original dataframe
    df_clean = db.copy()

    # Display NULL values information
    null_counts = df_clean.isnull().sum()
    null_percentage = (null_counts / len(df_clean)) * 100
    #print("\n=== Missing Values Percentage ===")
    #print(null_percentage[null_percentage > 0].round(2))

    # Drop columns with >= 70% null values
    df_clean = df_clean.dropna(axis=1, thresh=int(0.3 * len(df_clean)))

    # Drop rows with >= 70% null values
    df_clean = df_clean.dropna(axis=0, thresh=int(0.3 * df_clean.shape[1]))

    # Separate categorical and numerical columns
    cat_cols = df_clean.select_dtypes(include=['object']).columns  # FIXED: 'object' instead of 'category'
    num_cols = df_clean.select_dtypes(include=['number']).columns

    # Fill numerical nulls with median
    df_clean[num_cols] = df_clean[num_cols].fillna(df_clean[num_cols].median())

    # Fill categorical nulls with mode
    for col in cat_cols:
        df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0] if not df_clean[col].mode().empty else "missing")

    return df_clean

train_df = pd.read_csv('housing_price_train.csv', na_values='NA')
prepro_train_df = PreprocessingData(train_df)
#prepro_train_df.to_csv("train_preprocessed.csv", index=False)


### Correlations! (5 points)

* Now we have a clean dataset, but not all attributes are useful! 

* Display the corrlation between all features and the sales price. This will show you which feature affects sales price more. You may use *corr()* function. 

* Choose the most correlated features, and remove others. 

In [3]:
import pandas as pd

def corr(df):
    df_numeric = df.select_dtypes(include=['number'])
    correlation_matrix = df_numeric.corr()
    sales_price_corr = correlation_matrix["SalePrice"].sort_values(ascending=False)

    print("Feature Correlation with SalePrice:\n", sales_price_corr)

    # Select features with absolute correlation > 0.5 (excluding SalePrice)
    strong_features = sales_price_corr[abs(sales_price_corr) > 0.5].index

    if len(strong_features) <= 1:
        print("No strong correlations found. Returning original dataset.")
        return df_numeric  # Return unfiltered numeric data if no strong correlations

    df_filtered = df_numeric[strong_features]

    return df_filtered


train_corr = corr(prepro_train_df)
#train_corr.to_csv("train_corr.csv", index=False) if not train_corr.empty else print("No data to save.")





Feature Correlation with SalePrice:
 SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
MasVnrArea       0.472614
Fireplaces       0.466929
GarageYrBlt      0.466754
BsmtFinSF1       0.386420
LotFrontage      0.334771
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
K

## Part 2: Decision Tree (45 points)
#### Building a Decision Tree:
A Decision tree consists of nodes connected by edges. A decision tree is typically, a binary tree, which has the following properties:
- One node is marked as Root node
- Every node other than the root has a parent node
- Each node can have at most 2 child nodes (left edge & right edge)
- Leaf node is the node which contains pure data or when we reach to the maximum depth 

To create the decision tree model for scratch you need to create two classes (a class for the node, for example "class DecisionNode():" and a class for Decision Tree model, for example "class RegressionDecisionTree():")


1- DecisionNode class used to save some values for each node we do the spliting on it until we reach the leaf node
so we will save the following values for the node:
- feature: feature index.
- threshold: the value we used to split the data on.
- value: the average value for the leaf node.
- True_Branch: if the condition is true.
- False_Branch: if the condition is false.

In [31]:
class DecisionNode():
    def __init__(self, feature_idx=None, threshold=None, value=None, true_branch=None, false_branch=None):
        self.feature_idx = feature_idx # index of the feature that is used
        self.threshold = threshold     # threshold value for feature when making the decision
        self.value = value # Average value if the node is a leaf in the tree
        self.true_branch = true_branch # the node we go to if decision returns True
        self.false_branch = false_branch # the node we go to if decision returns False

# Decision Tree Class
This Class consists the following functions:
<ol>
<li> <b>build_tree</b>: used to create the decision tree nodes</li> 
<li> <b>calc_variance_reduction</b> : measure the impurity by using variance reduction measure (like MSE) </li> 
the function takes three parameters (parentRec: the records for the target before split,and the left and right records after splitting. This function used to measure the impurity for each node and decide if we will split or not.
<li> <b>majority_vote</b>: used to calculate values for the leaf nodes records which equal to the mean of these records.</li> 
<li><b>split_by_feature</b>: this function take the feature and the threshold and check if the feature is numerical so it split the records into two node (true which is the left edge and false which is the right edge)
if the feature is categorical so it split where the values equal to the threshold</li>
<li> <b>fit</b>: Used to train the dataset after spliting the data into two part x: features, y: target</li>
<li><b>predict_value</b>: used to predict the value for each record, it is a recursive method to find the leaf node that corresponds to prediction
<li><b>predict</b>: take all records for the test data and iterate into each record to predit the y(target) value and save the result into a prediction list. 

In [32]:
import pandas as pd
import numpy as np

class RegressionDecisionTree():
    # constructor
    def __init__(self, min_VarianceReduction=1e-7, max_depth=5):        
        self.root = None # root of this tree
        self.min_VarianceReduction = min_VarianceReduction # minimum VarianceReduction to allow splitting
        # used to stopping conductions
        self.max_depth = max_depth # maximum depth the tree grows to
 

    # used to create the decision tree nodes
    def build_tree(self, X, y, current_depth=0):
        # we will use decision dictionary to save the feature and the threshold we build the tree on 
        decision = None
        # we will use subtrees dictionary to save the feature and the threshold we build the tree on 
        subtrees = None
        largest_variance_Reduction = 0
        # add y as last column of X
        df = pd.concat((X, y), axis=1)
        n_rows, n_features = X.shape
        if current_depth <= self.max_depth:
            # iterate through every feature
            for feature_idx in range(n_features):
                # values of that column
                feature_values = X.iloc[:, feature_idx]                
                unique_values = feature_values.unique()                
                for threshold in unique_values:
                    X_trueEdge, X_falseEdge = self.split_by_feature(df, feature_idx, threshold)
                    if len(X_trueEdge) > 0 and len(X_falseEdge) > 0:
                        y_true = X_trueEdge.iloc[:,-1]
                        y_false = X_falseEdge.iloc[:,-1]                        
                        # Calculate impurity
                        VarianceRed = self.Calc_variance_reduction(y, y_true, y_false)
                        # Keep track of which feature gave the largest information gain
                        if VarianceRed > largest_variance_Reduction:
                            largest_variance_Reduction = VarianceRed
                            decision = {"feature_idx":feature_idx, "threshold":threshold}
                            subtrees = {"X_true":X_trueEdge.iloc[:,:-1],
                                        "y_true":y_true,
                                        "X_false":X_falseEdge.iloc[:,:-1],
                                        "y_false":y_false}

        # we will construct new branch of tree if the variance_Reduction is larger than minimum variance_Reduction that we've defined
        if largest_variance_Reduction > self.min_VarianceReduction:
            true_branch = self.build_tree(subtrees["X_true"], subtrees["y_true"], current_depth+1)
            false_branch = self.build_tree(subtrees["X_false"], subtrees["y_false"], current_depth+1)
            return DecisionNode(feature_idx=decision["feature_idx"], threshold=decision["threshold"], true_branch=true_branch, false_branch=false_branch)

        # at leaf node we calculate the mean for the records
        leaf_value = self.majority_vote(y)
        return DecisionNode(value=leaf_value)
                        
    # measure the impurity by using variance reduction measure (like MSE)
    # left_edgeRec= True edge: where condition is true
    # Right_edgeRec= False edge: where condition is false
    def Calc_variance_reduction(self, parentRec, left_edgeRec, Right_edgeRec):   
        # return the VarReduction = variance for parent - (Weight * var(leftEdge) + Weight * var(RightEdge)
        var_parent = np.var(parentRec)
        
        weight_left = len(left_edgeRec) / len(parentRec)
        weight_right = len(Right_edgeRec) / len(parentRec)
        var_left = np.var(left_edgeRec)
        var_right = np.var(Right_edgeRec)

        VarReduction = var_parent - (weight_left * var_left + weight_right * var_right)
        return VarReduction
        
    
    def majority_vote(self, Y): 
        # return the majority_vote for the leaf nodes 
        return Y.mean()
        
             
    def split_by_feature(self, db, feature_idx, threshold):
        # split the data into left_edge & right_edge depends one specified feature and the threshold
        left_edge = db[db.iloc [:, feature_idx] <= threshold]
        right_edge = db[db.iloc [:, feature_idx] > threshold]
        # return left & right edges
        return left_edge, right_edge

    
    # Used to train the dataset after spliting the data into x: features, y: target
    def fit(self, X, y):
        self.root = self.build_tree(X, y)


    def predict_value(self, xTest, tree=None):
        # recursive method to find the leaf node that corresponds to prediction
        if tree is None:
            tree = self.root
        
        # If it's a leaf node, return the value
        if tree.value is not None:
            return tree.value
        
        if xTest[tree.feature_idx] <= tree.threshold:
            return self.predict_value(xTest, tree.true_branch)
        else:
            return self.predict_value(xTest, tree.false_branch) 

    # to predict the value we need to pass the all records for features and we save the prediction for each records into a list
    def predict(self, XTest):
        y_pred = []
        for idx, row in XTest.iterrows():           
            y_pred.append(self.predict_value(row.values))
        return y_pred


- To Check the Accuracy for our prediction we use CalcAccuracy function which take the actual values for the test dataset and the predicted values and apply the RMSE formula.

In [33]:
import numpy as np
def CalcAccuracy(Actual_Y, Predicted_y):
    Actual_Y = np.array(Actual_Y)
    Predicted_y = np.array(Predicted_y)
    
    # Calculate the squared differences
    squared_diff = (Actual_Y - Predicted_y) ** 2
    
    # Calculate the mean of the squared differences
    mean_squared_diff = np.mean(squared_diff)
    
    # Take the square root to get RMSE
    rmse = np.sqrt(mean_squared_diff)
    
    return rmse

- Build decision tree model
- Fit the model
- Predict the values from test data 

In [34]:
import pandas as pd
import numpy as np

# Load the training and test data
train_data = pd.read_csv('housing_price_train.csv')
test_data = pd.read_csv('housing_price_test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# Preprocess the data 
train_data_clean = PreprocessingData(train_data)
test_data_clean = PreprocessingData(test_data)

# Select the strong features based on correlation with SalePrice
train_data_filtered = corr(train_data_clean)

# Separate the features (X) and target (y) for training
X_train = train_data_filtered.drop(columns=['SalePrice'])
y_train = train_data_filtered['SalePrice']

# Prepare the test data (features only, no target)
X_test = test_data_clean[X_train.columns]  # Ensure test data has the same features as training data

# Initialize and train the RegressionDecisionTree model
regression_tree = RegressionDecisionTree()
regression_tree.fit(X_train, y_train)

# Predict the values using the trained model
y_pred = regression_tree.predict(X_test)

# Merge the predictions with the sample_submission to compare with the actual SalePrice
predictions_df = pd.DataFrame({'Id': test_data['Id'], 'Predicted_SalePrice': y_pred})

# Merge the predictions with the sample_submission for RMSE calculation
merged_predictions = pd.merge(sample_submission, predictions_df, on='Id')

# Calculate RMSE (Root Mean Squared Error)
rmse = CalcAccuracy(merged_predictions['SalePrice'], merged_predictions['Predicted_SalePrice'])
mean_saleprice = np.mean(merged_predictions['SalePrice'])
relative_rmse = rmse / mean_saleprice
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Relative RMSE: {relative_rmse * 100:.2f}%")

#predictions_df.to_csv('predictions.csv', index=False)


Root Mean Squared Error (RMSE): 42621.33219017309
Relative RMSE: 23.98%


## Part 3: Random Forest (20 points)
#### Random forest class
- the Class consist of the following functions:
<ul>
    <li>Constructor: consists of the subset data (Training & Testing) dataset after preprocessing and a list of deciceion tree objects </li>    
    <li>Subsampling: Bagging we will take random sample with replacement for the Training dataset </li>
    <li>build_model: first make subsample for the training dataset, then split the data into featurespart(X) and targetpart(Y), then take 10 samples of the feature part, finally build the decision tree (fit), this function take the number of DT that we want to build</li>
    <li>predict: take the test dataset and make the prediction for the target field in all the tree in the random forest then take the mean for the prediction in each tree, finally add the mean of prediction to a list of predition </li>

</ul>

In [35]:
class RF(object):
    def __init__(self):
        self.Traindata = None  # training data set (loaded into memory)
        self.Testdata = None  # Test data set for prediction        
        self.trees = []  # list of decision trees 
          

     # This function generate a subsample with replacement
    def __subsampling(self, train_set, sample_size_ratio):       

        sample_number = round(len(train_set) * sample_size_ratio)
        return train_set.sample(n = sample_number, replace=True)

        
        
    def build_model(self, train_set, sample_size_ratio, number_of_trees):
        for i in range(number_of_trees):
            TrainingSample = self.__subsampling(train_set, sample_size_ratio)
            
            # Split the training data into features (X) and target (y)
            X_train = TrainingSample.drop(columns=['SalePrice'])  # Assuming 'SalePrice' is the target column
            y_train = TrainingSample['SalePrice']
            
            # build a tree
            tree = RegressionDecisionTree() 
            # Train the tree 
            tree.fit(X_train, y_train)  
            
            self.trees.append(tree)
            
               
    def predict(self, test_set):
        predictions = []
        for tree in self.trees:
            # Predict for each item in the list 
            X_test = test_set.drop(columns=['SalePrice'], errors='ignore')  # SalePrice is not in test data
            y_pred_tree = tree.predict(X_test)
            predictions.append(y_pred_tree)
        
        # Calculate the mean 
        predictions = np.array(predictions)
        mean_predictions = np.mean(predictions, axis=0)
        
        return mean_predictions

### Create Random Forest

* Create 10 Decision Tree in the randomforest
* Train the random forest with the dataset
* Use the created random forest to predict the test dataset 

In [36]:
import pandas as pd
# Load the training and test data
train_data = pd.read_csv('housing_price_train.csv')
test_data = pd.read_csv('housing_price_test.csv')

# Preprocess the data 
train_data_clean = PreprocessingData(train_data)
test_data_clean = PreprocessingData(test_data)

# Select the strong features based on correlation with SalePrice
train_data_filtered = corr(train_data_clean)

# Separate the features (X) and target (y) for training
X_train = train_data_filtered.drop(columns=['SalePrice'])
y_train = train_data_filtered['SalePrice']

# Prepare the test data (features only, no target)
#X_test = test_data_clean[X_train.columns]  # Ensure test data has the same features as training data

# Initialize the Random Forest model
rf_model = RF()

# Train the random forest with 10 decision trees, using 80% of data for sub sampling
rf_model.build_model(train_data_filtered, sample_size_ratio=0.8, number_of_trees=10)

# Predict the values using the trained random forest model
y_pred_rf = rf_model.predict(X_test)

# Create the predictions DataFrame
predictions_df = pd.DataFrame({'Id': test_data['Id'], 'Predicted_SalePrice': y_pred_rf})

# Merge predictions with the sample_submission for RMSE calculation
merged_predictions = pd.merge(sample_submission, predictions_df, on='Id')
rmse = CalcAccuracy(merged_predictions['SalePrice'], merged_predictions['Predicted_SalePrice'])
mean_sale_price = np.mean(merged_predictions['SalePrice'])
relative_rmse = rmse / mean_sale_price
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Relative RMSE: {relative_rmse * 100:.2f}%")


#predictions_df.to_csv('rf_predictions.csv', index=False)

Root Mean Squared Error (RMSE): 38390.22841121438
Relative RMSE: 21.60%


##Part4: Comparison! (15 points)

Now that you have finished implementing your Random Forest, it's time for some experiments and analysis! 

* Use the Random Forest in the scikit-learn library and train it on the same dataset. 

* Compare the accuracy given by your Random Forest to the scikit-learn one. 

* Increase the number of trees in your Random Forest. Does it improve the accuracy? 

* Make a table for comparing your Random Forest accuracy with different number of trees with the scikit-learn one. What is your conclusion? 

In [50]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor  

# Load the dataset
train_data = pd.read_csv('housing_price_train.csv')
test_data = pd.read_csv('housing_price_test.csv')
sample_submission = pd.read_csv('sample_submission.csv') 

# Preprocess the data 
train_data_clean = PreprocessingData(train_data)
test_data_clean = PreprocessingData(test_data)

# Select strong features
train_data_filtered = corr(train_data_clean)

# Split into features and target
X_train = train_data_filtered.drop(columns=['SalePrice'])
y_train = train_data_filtered['SalePrice']
X_test = test_data_clean[X_train.columns]  # Ensure test data has the same features

# List of different tree counts to test
tree_counts = [1, 5, 10, 15, 20]

# Store results
results = []

# Run for different tree counts
for num_trees in tree_counts:
    rf_model = RF()
    rf_model.build_model(train_data_filtered, sample_size_ratio=0.8, number_of_trees=num_trees)
    y_pred_rf = rf_model.predict(X_test)
    y_pred_rf = y_pred_rf[:len(sample_submission)]

    # RF
    if "SalePrice" in sample_submission.columns:
        rmse_rf = CalcAccuracy(sample_submission['SalePrice'], y_pred_rf)
        results.append(("RF", num_trees, rmse_rf))
    else:
        print("Error: 'SalePrice' column missing in sample_submission.")

    # scikit-learn
    try:
        sk_rf_model = RandomForestRegressor(n_estimators=50, random_state=42)
        sk_rf_model.fit(X_train, y_train)
        y_pred_sk_rf = sk_rf_model.predict(X_test)
        y_pred_sk_rf = y_pred_sk_rf[:len(sample_submission)]
        rmse_sk_rf = CalcAccuracy(sample_submission['SalePrice'], y_pred_sk_rf)
        results.append(("Sklearn", num_trees, rmse_sk_rf))
    except ModuleNotFoundError:
        print("Error: scikit-learn is not installed.")

# Calculate the difference in RMSE between RF and sklearn models
rmse_diff = []
for result_rf, result_sk in zip(results[::2], results[1::2]):  # Pair up RF and sklearn results
    num_trees_rf, _, rmse_rf_value = result_rf
    num_trees_sk, _, rmse_sk_value = result_sk
    rmse_diff.append((num_trees_rf, rmse_rf_value - rmse_sk_value))

# Display RMSE comparison
print("Comparison of RMSE for different tree counts:")
print(pd.DataFrame(results, columns=["Model", "Number of Trees", "RMSE"]))

# Display RMSE difference
print("Difference in RMSE between RF and Sklearn:")
print(pd.DataFrame(rmse_diff, columns=["Number of Trees", "RMSE Difference"]))




Comparison of RMSE for different tree counts:
     Model  Number of Trees          RMSE
0       RF                1  44059.336131
1  Sklearn                1  37544.002866
2       RF                5  38897.349281
3  Sklearn                5  37544.002866
4       RF               10  38415.890606
5  Sklearn               10  37544.002866
6       RF               15  38217.491259
7  Sklearn               15  37544.002866
8       RF               20  38344.172911
9  Sklearn               20  37544.002866
Difference in RMSE between RF and Sklearn:
  Number of Trees  RMSE Difference
0              RF      6515.333266
1              RF      1353.346415
2              RF       871.887740
3              RF       673.488394
4              RF       800.170045
